# Train segmentation model for DONKEY
This notebook trains the segmentation model

## Usage:
Run all cells

## Requirements:
./content/datasets/processed_data/sim_rectified \
./content/datasets/processed_data/sim_rectified_mask_retouched \
./content/datasets/processed_data/real_rectified \
./content/datasets/processed_data/real_rectified_mask_retouched

## Outputs:
./content/segmentation_checkpoints/Model_weights.hdf5

In [ ]:
import h5py
import numpy as np
import PIL.Image as Image
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint
import h5py
import numpy as np
import PIL.Image as Image

import os
import cv2
import matplotlib.pyplot as plt
import os
from PIL import Image
import matplotlib.image as mpimg
from PIL import Image, ImageOps

import re
from collections import OrderedDict

img = np.array(Image.open("./content/datasets/data_in/objects/real/26_cam-image_array_.jpg"))
obj_ids = np.unique(img)
print(obj_ids)

def natural_sort_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split(r'(\d+)', s)]

def load_data(images_folder):
  raw_image=[]
  image_files = sorted(os.listdir(images_folder),key=natural_sort_key) 
  for image_file in image_files:
      if image_file[0]!=".":
          image = mpimg.imread(images_folder+image_file)
          if (image_file.split(".")[-1]=="png"):
              image=image[:,:,:3]
              image = (image * 255).astype(np.uint8)
          raw_image.append(image)
  return raw_image


def load_data_mask(images_folder):
  raw_image=[]
  image_files = sorted(os.listdir(images_folder),key=natural_sort_key)
  #print(image_files)    
  for image_file in image_files:
      if image_file[0]==".":
          break
      image = mpimg.imread(images_folder+image_file)
      if (image_file.split(".")[-1]=="png"):
          image = (image * 255).astype(np.uint8)
      raw_image.append(image)
  return raw_image


raw_image_sim = []
raw_image_real = []

print("-STARTING sim- ")
images_folder = f"./content/datasets/processed_data/sim_rectified/"
raw_image_sim=load_data(images_folder)
images_folder = f"./content/datasets/processed_data/sim_rectified_mask_retouched/"
raw_mask_sim=load_data_mask(images_folder)

In [3]:
print("-STARTING real- ")
images_folder = f"./content/datasets/processed_data/real/"
raw_image_real=load_data(images_folder)
images_folder = f"./content/datasets/processed_data/real_mask_retouched/"
raw_mask_real=load_data_mask(images_folder)

-STARTING real- 


In [ ]:
print(len(raw_image_sim))
print(len(raw_mask_sim))
for i in range(0, 100,1):
      print(i)
      # Create a 2x2 grid of subplots
      fig, axs = plt.subplots(1, 2, figsize=(15, 6))
      plt.subplots_adjust(wspace=0.2, hspace=0.4)

      # Plot the real and fake images side by side in the first row
      axs[0].imshow(raw_image_sim[i])
      axs[0].set_title('Sim Image')
      axs[1].imshow(raw_mask_sim[i])
      axs[1].set_title('Mask Image')
      for ax in axs.flat:
          ax.set_xticks([])
          ax.set_yticks([])

      # Show the plot for the current iteration
      plt.show()

In [ ]:
print(len(raw_image_real))
print(len(raw_mask_real))
for i in range(0, 100,1):
      print(i)
      # Create a 2x2 grid of subplots
      fig, axs = plt.subplots(1, 2, figsize=(15, 6))
      plt.subplots_adjust(wspace=0.2, hspace=0.4)

      # Plot the real and fake images side by side in the first row
      axs[0].imshow(raw_image_real[i])
      axs[0].set_title('Sim Image')
      axs[1].imshow(raw_mask_real[i])
      axs[1].set_title('Mask Image')
      for ax in axs.flat:
          ax.set_xticks([])
          ax.set_yticks([])

      # Show the plot for the current iteration
      plt.show()

In [6]:
import tensorflow as tf
import numpy as np

def map_to_desired_structure(input_image, output_image):
    return {'pixel_values': tf.cast(input_image, tf.float32), 'labels': tf.cast(output_image, tf.float32)}

def map_values(value):
    if value in [128]:
        return 1
    elif value in [255]:
        return 2
    else:
        return 0
    return value
def create_tf_dataset(input_dict, output_dict, input_indices):
    # Make sure input_indices and output_indices have the same length

    # Create lists to store input and output images
    input_images = []
    output_images = []

    # Iterate over the indices and extract the corresponding images

    print("Elements number: ",len(input_indices))
    for index in input_indices:
        if index < len(input_dict):
            input_images.append(input_dict[index])


    vectorized_map = np.vectorize(map_values)
    for index in input_indices:
        if index < len(output_dict):
            label_value = output_dict[index]
            output_images.append(vectorized_map(label_value))
    print("Processing")
    # Convert lists to numpy arrays
    input_images = np.array(input_images)
    # input_images = tf.reverse(input_images, axis=[-1])
    input_images = tf.transpose(input_images, (0,3, 1, 2))
    output_images = np.array(output_images)

    # Create TensorFlow dataset from numpy arrays
    dataset = tf.data.Dataset.from_tensor_slices((input_images, output_images))
    print("Mapping")

    dataset = dataset.map(map_to_desired_structure)

    return dataset

In [ ]:
train_indexes_gan = {}
test_indexes_gan = {}


pattern = 'ttvttvttvt'
pattern_pointer = 0
train_dataset_indices_inner = []
test_dataset_indices_inner = []

for index in range(0, min(len(raw_image_sim),len(raw_mask_sim))):
    pattern_char = pattern[pattern_pointer]
    if pattern_char == 't':
        train_dataset_indices_inner.append(index)
    else:
        test_dataset_indices_inner.append(index)
    pattern_pointer = (pattern_pointer + 1) % len(pattern)
train_indexes=train_dataset_indices_inner
test_indexes=test_dataset_indices_inner

print("GAN train and test")
print("Train: ",len(train_indexes))
print("Test: ",len(test_indexes))

In [ ]:
tf_dataset_train_sim = create_tf_dataset(raw_image_sim, raw_mask_sim, train_indexes)
tf_dataset_test_sim = create_tf_dataset(raw_image_sim, raw_mask_sim, test_indexes)

In [ ]:
tf_dataset_train_real = create_tf_dataset(raw_image_real, raw_mask_real, train_indexes)
tf_dataset_test_real = create_tf_dataset(raw_image_real, raw_mask_real, test_indexes)

In [ ]:
import tensorflow_datasets as tfds

In [11]:
import tensorflow as tf
from tensorflow.keras import backend

image_size = 512
mean = tf.constant([0.485, 0.456, 0.406])
std = tf.constant([0.229, 0.224, 0.225])


def normalize(input_image, input_mask):
    input_image = tf.image.convert_image_dtype(input_image, tf.float32)
    input_image = (input_image - mean) / tf.maximum(std, backend.epsilon())
    input_mask -= 1
    return input_image, input_mask


def load_image(datapoint):
    input_image = tf.image.resize(datapoint["image"], (image_size, image_size))
    input_mask = tf.image.resize(
        datapoint["segmentation_mask"],
        (image_size, image_size),
        method="bilinear",
    )

    input_image, input_mask = normalize(input_image, input_mask)
    input_image = tf.transpose(input_image, (2, 0, 1))
    return {"pixel_values": input_image, "labels": tf.squeeze(input_mask)}


In [12]:
train_ds =tf_dataset_train_sim
test_ds =tf_dataset_test_sim
train_ds2 =tf_dataset_train_real
test_ds2 =tf_dataset_test_real

In [13]:
auto = tf.data.AUTOTUNE
batch_size = 4

train_ds = (
    train_ds
    .shuffle(batch_size * 10)
    .batch(batch_size)
    .prefetch(auto)
    .cache()
)

test_ds = (
    test_ds
    .shuffle(batch_size * 10)
    .batch(batch_size)
    .prefetch(auto)
    .cache()
)

train_ds2 = (
    train_ds2
    .shuffle(batch_size * 10)
    .batch(batch_size)
    .prefetch(auto)
    .cache()
)

test_ds2 = (
    test_ds2
    .shuffle(batch_size * 10)
    .batch(batch_size)
    .prefetch(auto)
    .cache()
)

In [14]:
print(train_ds.element_spec)
print(test_ds.element_spec)
print(test_ds2.element_spec)

{'pixel_values': TensorSpec(shape=(None, 3, 140, 320), dtype=tf.float32, name=None), 'labels': TensorSpec(shape=(None, 140, 320), dtype=tf.float32, name=None)}
{'pixel_values': TensorSpec(shape=(None, 3, 140, 320), dtype=tf.float32, name=None), 'labels': TensorSpec(shape=(None, 140, 320), dtype=tf.float32, name=None)}
{'pixel_values': TensorSpec(shape=(None, 3, 140, 320), dtype=tf.float32, name=None), 'labels': TensorSpec(shape=(None, 140, 320), dtype=tf.float32, name=None)}


In [ ]:
def display(display_list):
    plt.figure(figsize=(15, 15))

    title = ["Input Image", "True Mask", "Predicted Mask"]

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i + 1)
        plt.title(title[i])
        print(display_list[i].shape)
        plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
        plt.axis("off")
    plt.show()


for samples in train_ds2.take(10):
    sample_image, sample_mask = samples["pixel_values"][0], samples["labels"][0]
    sample_image = tf.transpose(sample_image, (1, 2, 0))
    sample_mask = tf.expand_dims(sample_mask, -1)
    display([sample_image, sample_mask])

In [ ]:
from transformers import TFSegformerForSemanticSegmentation

model_checkpoint = "nvidia/mit-b0"
id2label = {0: "not_important", 1: "road", 2: "marking"}
label2id = {label: id for id, label in id2label.items()}
num_labels = len(id2label)
model = TFSegformerForSemanticSegmentation.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

In [17]:
lr = 0.00006
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
model.compile(optimizer=optimizer)

In [18]:
from IPython.display import clear_output


def create_mask(pred_mask):
    pred_mask = tf.math.argmax(pred_mask, axis=1)
    pred_mask = tf.expand_dims(pred_mask, -1)
    return pred_mask[0]


def show_predictions(dataset=None, num=1):
    if dataset:
        for sample in dataset.take(num):
            images, masks = sample["pixel_values"], sample["labels"]
            masks = tf.expand_dims(masks, -1)
            pred_masks = model.predict(images).logits
            images = tf.transpose(images, (0, 2, 3, 1))
            display([images[0], masks[0], create_mask(pred_masks)])
    else:
        display(
            [
                sample_image,
                sample_mask,
                create_mask(model.predict(tf.expand_dims(sample_image, 0))),
            ]
        )


class DisplayCallback(tf.keras.callbacks.Callback):
    def __init__(self, dataset, **kwargs):
        super().__init__(**kwargs)
        self.dataset = dataset

    def on_epoch_end(self, epoch, logs=None):
        #clear_output(wait=True)
        show_predictions(self.dataset)
        print("\nSample Prediction after epoch {}\n".format(epoch + 1))

In [19]:
checkpoint_file_path2 = "./content/segmentation_checkpoints/Model_weights.hdf5"
model.load_weights(checkpoint_file_path2)

In [ ]:
epochs = 2
for i in range(0,3):
    print("SIM")
    history = model.fit(
        train_ds,
        validation_data=test_ds,
        callbacks=[DisplayCallback(test_ds)],
        epochs=epochs,
    )
    print("REAL")
    history = model.fit(
        train_ds2,
        validation_data=test_ds2,
        callbacks=[DisplayCallback(test_ds2)],
        epochs=epochs,
    )

In [21]:
checkpoint_file_path = "./content/segmentation_checkpoints/Model_weights.hdf5"
model.save_weights(checkpoint_file_path)

In [ ]:
show_predictions(test_ds2, 50)

REWATCH

In [54]:
checkpoint_file_path2 = "./content/segmentation_checkpoints/Model_weights.hdf5"
model.load_weights(checkpoint_file_path2)

In [55]:
def savepred(display_list,id,domain):
    plt.figure(figsize=(15, 15))

    title = ["Input Image", "True Mask", "Predicted Mask"]

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i + 1)
        plt.title(title[i])
        print(display_list[i].shape)
        plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
        plt.axis("off")
    save_path="./content/output_plots/predicted_mask_"+str(domain)+"/img_"+str(id)+".png"
    plt.savefig(save_path)
    # plt.show()

In [56]:
def save_predictions(domain,dataset=None, num=1):
    if dataset:
        for i,sample in enumerate(dataset.take(num)):
            images, masks = sample["pixel_values"], sample["labels"]
            masks = tf.expand_dims(masks, -1)
            pred_masks = model.predict(images).logits
            images = tf.transpose(images, (0, 2, 3, 1))
            savepred([images[0], masks[0], create_mask(pred_masks)],i,domain)
    else:
        savepred(
            [
                sample_image,
                sample_mask,
                create_mask(model.predict(tf.expand_dims(sample_image, 0))),
            ]
        )

In [ ]:
save_predictions("sim",test_ds, 1000)

In [ ]:
save_predictions("real",test_ds2, 1000)